---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing

[Certificate](https://www.coursera.org/account/accomplishments/verify/C9QNCY7JN5HQ)

This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [3]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    archivo = open("university_towns.txt","r") #Abre el archivo
    state="" #Inicializa la variable state donde se va a validar el estado que se esta revisando
    answer=[] #Inicializa la variable answer como una lista vacia
    #print(archivo.readlines())
    for linea in archivo.readlines():     #Ciclo para leer todas las lineas del archivo
        linea = linea.replace("\n","")  #Elimina el caracter "\n" de todas las lineas
        if linea.find("[edit]") != -1:  #Verifica si el nombre en la linea es un estado
            state = linea.replace("[edit]","") #Elimina el substring "[edit]" del nombre y guarda el estado actual
        else:
            linea = linea.split(" (")[0]    #Elimina el resto del nombre de la region, todo lo que esta despues de " ("
            answer.append([state,linea])    #Agrega el estado y la region a la lista
    answer = pd.DataFrame(answer,columns=["State","RegionName"])  #Convierte la lista en un data frame
    #print(answer)
    archivo.close() #Cierra el archivo abierto
    return answer
#print(get_list_of_university_towns())

In [4]:
def get_gdplev():
    '''Return a data frame with the quarter information of the file gdplev.xls'''
    archivo = pd.read_excel("gdplev.xls",header=None,names=["Quarter","GDP(Current)","GDP(2009)"],usecols=[4,5,6],skiprows=220)
    #Abre el archivo de excel y crea un data frame con las columnas solicitadas
    #print(archivo)
    return archivo
#print(get_gdplev())

def get_recession_start(flag=0):
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    '''Flag es un parametro opcional que determina que retorna la funcion al final de su ejecución'''
    df = get_gdplev() #Abre el dataframe del archivo gdplev.xls con los filtros respectivos ya aplicados
    decreciente = 0 #Inicializador del contador que indica cuantos valores decrecientes van continuos
    for index, row in df.iterrows(): #Ciclo for que itera sobre todas las filas del data frame
        if index != 0:  #Valida que no sea la primera fila del data frame
            if df["GDP(Current)"].iloc[index] < df["GDP(Current)"].iloc[index-1]: #Valida si el GDP de este periodo es menor al GDP del periodo anterior
                decreciente = decreciente + 1 #Suma una unidad a los periodos decrecientes continuos
            else:
                decreciente = 0 #Devuelve a cero el contador
            if decreciente >= 2: #Valida si ya hay dos periodos decrecientes continuos
                #print("Periodo encontrado")
                break #Sale del ciclo for
    if flag == 0:
        return df["Quarter"].iloc[index-2] #Regresa el periodo en el que comenzo la recesion 
    elif flag == 1:
        return df.iloc[index-2:] #Retorna el dataframe unicamente desde el inicio de la recesion
    else:
        return index-2 #Retorna el indice del dataframe donde comenzo la recesion 
#print(get_recession_start())

In [5]:
def get_recession_end(flag=0):
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    '''Flag es un parametro opcional que determina que retorna la funcion al final de su ejecución'''
    df = get_recession_start(flag=1).reset_index(drop=True) #Abre el dataframe del archivo gdplev.xls mostrado unicamente a partir del inicio de la recesion
    #Resetea el index del dataframe desde cero y no guarda el index anterior como una columna
    #print(df)
    creciente = 0 #Inicializador del contador que indica cuantos valores crecientes van continuos
    for index, row in df.iterrows(): #Ciclo for que itera sobre todas las filas del data frame
        if index != 0: #Valida que no sea la primera fila del data frame
            if df["GDP(Current)"].iloc[index] > df["GDP(Current)"].iloc[index-1]: #Valida que el GDP del periodo actual sea mayor al GDP del periodo anterior
                creciente = creciente + 1 #Aumenta el valor de creciente
            else:
                creciente = 0 #Reinicia la variable creciente
            if creciente >= 2: #Valida que ya se hallan encontrado 2 periodos creicentes seguidos
                break #Sale del ciclo for
    if flag == 0:
        return df["Quarter"].iloc[index] #Retorna el periodo en el que se acabo la recesion 
    else:
        return index #Retorna el indice del dataframe (El dataframe recortado, el cual comienza en el inicio de la recesion) donde acaba la recesion
#print(get_recession_end(flag=0))

In [6]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    inicio = get_recession_start(flag=2) #Indice donde comienza la recesion 
    #print(inicio)
    final = inicio + get_recession_end(flag=1) +1#Indice donde termina la recesion
    #print(final)
    df = get_gdplev().iloc[inicio:final] #Abre el dataframe con los datos unicamente de la recesion
    #print (df)
    return df["Quarter"].loc[np.argmin(df["GDP(Current)"])] #Retorna el periodo en el cual el GDP fue minimo
#print(get_recession_bottom())

In [7]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    df=pd.read_csv("City_Zhvi_AllHomes.csv") #Crea un dataframe a partir de un archivo CSV
    #print(df.head())
    aux=df.drop(labels=["RegionID","RegionName","State","Metro","CountyName","SizeRank"],axis=1) #Crea un nuevo dataframe llamado aux que es una copia del dataframe df pero eliminando las columnas mencionadas
    columnas=list(aux.columns) #Crea una lista con los nombres de las columnas del dataframe aux
    for columna in columnas: #Ciclo for para eliminar todos los datos de años inferiores al 2000
        if int(columna[:4]) < 2000: #Verifica que el año de ese dato sea menor al 2000
            aux = aux.drop(labels=columna,axis=1) #Elimina la columna del data frame aux
            df = df.drop(labels=columna,axis=1) #Elimina la columna del data frame df
        else:
            break #Termina el ciclo si ya se periodos los años inferiores a 2000
    columnas=list(aux.columns) #actualiza la lista con los nombres de las columnas del dataframe aux
    current_year="2000" #Inicializa la variable 
    current_quarter=1 #inicializa la variable
    mes = 0 #inicializa la variable
    periodos = [] #Inicializa la lista que guardara los periodos que se incluiran en ese cuarto
    cuartos = []
    for columna in columnas: #Ciclo for que crea los cuartos
        #print("run")
        mes = int(columna.split("-")[1]) #Variable que indica el mes del periodo actual
        if mes <= 3: #Asigna a quarter el valor que le corresponda segun el mes
            quarter = 1
        elif mes <= 6:
            quarter = 2
        elif mes <= 9:
            quarter = 3
        else:
            quarter = 4
        if quarter == current_quarter: #Verifica que aun no se cambiara de periodo
            periodos.append(columna) #Si es el mismo cuarto, agrega el periodo a columnas
        else:
            df[current_year+"q"+str(current_quarter)]=df[periodos].apply(np.mean,axis=1) 
            #Crea una nueva columna en el dataframe df con el nombre del cuarto y la media de este
            cuartos.append(current_year+"q"+str(current_quarter)) #Agrega el nuevo cuarto guardado a la lista cuartos
            current_year=columna.split("-")[0] #Le asigna un nuevo valor al año
            current_quarter = quarter #Le asigna un nuevo valor a current quarter
            periodos = [] #Vacia nuevamente la lista periodos
            periodos.append(columna) #Agrega el periodo actual a periodos
    df[current_year+"q"+str(current_quarter)]=df[periodos].apply(np.mean,axis=1) 
    #Crea una nueva columna en el dataframe df con el nombre del ultimo cuarto y la media de este
    cuartos.append(current_year+"q"+str(current_quarter)) #Agrega el nuevo cuarto guardado a la lista cuartos
    df = df[["State","RegionName"]+cuartos].set_index(["State","RegionName"]) #Separa las columnas requeridas del data frame y asigna los indices pedidos
    #print(columnas)
    #print(df.head())
    return df
#print(convert_housing_data_to_quarters())
#housingPrices_aux=convert_housing_data_to_quarters() 
'''No olvidar comentar esta linea'''
pass


'No olvidar comentar esta linea'

In [27]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    anno_inicial=int(get_recession_start().split("q")[0]) #Toma el año en el cual comienza la recesion
    periodo_inicial=int(get_recession_start().split("q")[1]) #Toma el numero de periodo en el que comienza la recesion
    if periodo_inicial == 1: #Verifica si el periodo inicial es 1
        anno_inicial -= 1 #Le resta uno al año
        periodo_inicial = 4 #Asigna el periodo actual como 4
    else:
        periodo_inicial -= 1 #Le resta uno al periodo
    periodo_inicial="q".join([str(anno_inicial),str(periodo_inicial)]) #Asigna el nombre del cuarto anterior al que inicia la recesion a periodo_inicial
    periodo_final=get_recession_bottom() #Guarda el periodo mas bajo de la recesion
    housingPrices=convert_housing_data_to_quarters()[[periodo_inicial,periodo_final]] #Obtiene los dos periodos requeridos de housing prices
    #housingPrices=housingPrices_aux[[periodo_inicial,periodo_final]] 
    '''Linea auxiliar para no tener que correr la funcion cada vez que se realiza una'''
    housingPrices["Ratio"]=housingPrices[periodo_inicial]/housingPrices[periodo_final] # Calcula el ratio
    housingPrices = housingPrices["Ratio"].reset_index().replace({"State":states}) #.set_index(["State","RegionName"])
    #Deja unicamente la columna de ratio y reemplaza las abreviaturas de los estados por sus nombre completos y reinicia el index del data set
    housingPricesUniversityTowns= get_list_of_university_towns().merge(housingPrices,how="left",left_on=["State","RegionName"],right_on=["State","RegionName"])
    #Crea el dataset haciendo housingPricesUniversityTowns un left outter join para tener un dataset que tenga solo los ratios de los university towns
    #print(housingPrices.head())
    #print(housingPricesUniversityTowns.head())
    house_index=housingPrices.set_index(["State","RegionName"]).index #Obtiene los estados y las regiones de todo el archivo de housing prices
    universityTown_index=get_list_of_university_towns().set_index(["State","RegionName"]).index #Obtiene los estados y las regiones de todos los university towns
    mask = ~house_index.isin(universityTown_index) #Genera una mascara booleana que indica False si el estado y region de housingPrices es un university town y true de lo contrario (Notese que el caracter ~ sirve para negar)
    housingPricesNonUniversityTowns = housingPrices.loc[mask] #Muestra solo las filas que tengan valor true (Las que no son university towns)
    #print(housingPricesNonUniversityTowns.head())
    aux, p = ttest_ind(housingPricesUniversityTowns["Ratio"],housingPricesNonUniversityTowns["Ratio"],nan_policy="omit")
    different=False
    if p < 0.01:
        different=True
    better = "non-university town"
    if np.mean(housingPricesUniversityTowns["Ratio"]) < np.mean(housingPricesNonUniversityTowns["Ratio"]):
        better = "university town"
    return different,p,better
print(run_ttest())
'''
Codigo de ejemplo para eliminar los valores repetidos de un dataframe (Valores de un dataframe que ya estan en otro)
a_aux = pd.DataFrame({0:[1,2,3],1:[10,20,30]})
b_aux = pd.DataFrame({0:[0,1,2,3],1:[0,1,20,3]})
print(a_aux)
print(b_aux)
a_index = a_aux.set_index([0,1]).index
b_index = b_aux.set_index([0,1]).index
print(a_index)
print(b_index)
mask = ~a_index.isin(b_index)
print(mask)
result = a_aux.loc[mask]
print(result)
'''
pass

(True, 0.0027240637047531249, 'university town')
